In [5]:
from prometheus_api_client import PrometheusConnect,  MetricSnapshotDataFrame, MetricRangeDataFrame
from prometheus_api_client.utils import parse_datetime
import pandas as pd
import datetime as dt

prom = PrometheusConnect(url ="http://10.214.241.226:39090/", disable_ssl=True)

# Get the list of all the metrics that the Prometheus host scrapes
# prom.all_metrics()

In [6]:
def range_data_query(sql):
    data = prom.custom_query(sql)
    return data

def add_data2dict1(data, sql, tag, fc):
    dd = range_data_query(sql)

    for d in dd:
        pod_name = d["metric"]["pod"]
        value = float(d["value"][1])
        if pod_name in data:
            data[pod_name][tag] = fc(value)
    

In [7]:

data_total_dict = dict()
course_label = "instance"
node_label = "kubernetes_node"

data = range_data_query("last_over_time(kube_pod_labels[15d])")
for d in data:
    d = d["metric"]
    if course_label in d:
        dd = dict()
        dd["label"] = d[course_label]
        if node_label in d:
            dd["node"] = d[node_label]
        data_total_dict[d["pod"]] = dd

# print(data_total_dict)


In [8]:


sql_list = [
    ("sum(stddev_over_time(node_namespace_pod_container:container_cpu_usage_seconds_total:sum_irate[15d])) by (pod)", "cpu_std/m", lambda x : x * 1000),
    ("sum(avg_over_time(node_namespace_pod_container:container_cpu_usage_seconds_total:sum_irate[15d])) by (pod)", "cpu_avg/m", lambda x: x * 1000),
    ("sum(last_over_time(cluster:namespace:pod_cpu:active:kube_pod_container_resource_requests[15d])) by (pod)", "cpu_request/m", lambda x: x * 1000),
    ("sum(last_over_time(cluster:namespace:pod_cpu:active:kube_pod_container_resource_limits[15d])) by (pod)", "cpu_limit/m", lambda x: x * 1000),


    ("sum(stddev_over_time(node_namespace_pod_container:container_memory_working_set_bytes[15d])) by (pod)", "mem_std/MiB", lambda x: x / 1024/ 1024),
    ("sum(avg_over_time(node_namespace_pod_container:container_memory_working_set_bytes[15d])) by (pod)", "mem_avg/MiB", lambda x: x / 1024 / 1024),
    ("sum(last_over_time(cluster:namespace:pod_memory:active:kube_pod_container_resource_requests[15d])) by (pod)", "mem_request/MiB", lambda x: x / 1024 / 1024),
    ("sum(last_over_time(cluster:namespace:pod_memory:active:kube_pod_container_resource_limits[15d])) by (pod)", "mem_limit/MiB", lambda x: x / 1024 / 1024),


]

for sql in sql_list:
    add_data2dict1(data_total_dict, sql[0], sql[1], sql[2])


print(data_total_dict)

{'random-load-job-6bdwz': {'label': '10.244.1.13:8443', 'cpu_std/m': 67.39161675999044, 'cpu_avg/m': 350.62746846000005, 'cpu_request/m': 200.0, 'cpu_limit/m': 400.0, 'mem_std/MiB': 145.63456821687905, 'mem_avg/MiB': 327.25, 'mem_request/MiB': 400.0, 'mem_limit/MiB': 800.0}, 'random-load-job-6bghb': {'label': '10.244.1.13:8443', 'cpu_request/m': 200.0, 'cpu_limit/m': 400.0, 'mem_request/MiB': 400.0, 'mem_limit/MiB': 800.0}, 'kube-flannel-ds-hx96m': {'label': '10.244.1.13:8443', 'cpu_std/m': 2.2280827669232957, 'cpu_avg/m': 9.549956316110842, 'cpu_request/m': 100.0, 'cpu_limit/m': 100.0, 'mem_std/MiB': 0.41735513041440575, 'mem_avg/MiB': 24.451808055311666, 'mem_request/MiB': 50.0, 'mem_limit/MiB': 50.0}, 'kube-flannel-ds-sn8r6': {'label': '10.244.1.13:8443', 'cpu_std/m': 2.6170711819398007, 'cpu_avg/m': 11.803393722805282, 'cpu_request/m': 100.0, 'cpu_limit/m': 100.0, 'mem_std/MiB': 0.19385781416350967, 'mem_avg/MiB': 14.028566953833774, 'mem_request/MiB': 50.0, 'mem_limit/MiB': 50.0},

In [9]:
data = pd.DataFrame(data_total_dict)

data = data.T
data.to_csv("pod_info.csv")

In [10]:
statistics_label = ["cpu_std/m", "cpu_avg/m", "mem_std/MiB", "mem_avg/MiB"]

new_data = data.groupby(data["label"])[statistics_label].mean()

new_data.loc["all"] = data[statistics_label].mean()
new_data.to_csv("statistics.csv")
